# Logistic equation

## Euler method

In [ ]:
# Compute the derivative of N(t)
deriv_Nt(nt, K, r) = r*nt*(1-nt/K)

# function deriv_Nt(nt::Float64, K::Float64, r::Float64)::Float64
#     return r*nt*(1-nt/K)
# end

In [ ]:
methods(deriv_Nt)

In [ ]:
# Compute N(t + h)
next_Nt(nt, K, r, h) = nt + h*deriv_Nt(nt, K, r)

In [ ]:
# 各種パラメータを定義
tmax = 10.0 # 終了時刻
r = 1.0 # パラメータ1
K = 10.0 #パラメータ2
N0 = 1.0 # 人口の初期値
numtimes  = 100 # 0 ≦ t < tmaxの分割数

h = tmax/numtimes

In [ ]:
# 型安定性のチェック
@code_warntype deriv_Nt(N0, K, r)

In [ ]:
# 型安定性のチェック
@code_warntype deriv_Nt(N0, K, r)

In [ ]:
results = Vector{Float64}(undef, numtimes+1)

# 初期値
results[1] = N0

# 時間発展
for t in 1:numtimes
    results[t+1] = next_Nt(results[t], K, r, h)
end

In [ ]:
using Plots

times = LinRange(0, tmax, numtimes+1)

plot(times, results, label="Numerical", xlabel="time")

In [ ]:
exact_Nt(time::Float64) = K/(1+(K-N0) / N0 * exp(-r*time))

In [ ]:
# broadcast

exact_Nt.(times)

In [ ]:
err = abs.(results .- exact_Nt.(times))

# 引き算はbroadcastしなくても配列-配列してくれるが、absはもともとは配列を引数にできないのでbroadcast

p = plot(yaxis=:log, ylims=(1e-10, 1000),  xlabel="time")
plot!(p, times, results, label="Numerical")
plot!(p, times, err, marker=:x, label="Error")

## DifferentialEquations.jl

In [ ]:
using DifferentialEquations

# ロジスティック方程式の定義
function logistic!(du, u, params, t)
    r, K = params
    du[1] = r * u[1] * (1 - u[1]/K)
end

# パラメータの設定
params = (r, K)

# 初期条件の設定
u0 = [1.0]  # 開始時点の人口サイズ

# 時間範囲の設定
tspan = (0.0, tmax)

# 問題の設定
prob = ODEProblem(logistic!, u0, tspan, params)

# 微分方程式の解 (5次のルンゲクッタ法)
sol = solve(prob, Tsit5(), abstol=1e-8, reltol=1e-8)
;

In [ ]:
sol.t

In [ ]:
sol.u

In [ ]:
# 解のプロット

results_de = [u_[1] for u_ in sol.u]

p = plot(yaxis=:log, ylims=(1e-10, 1000),  xlabel="time")
plot!(p, sol.t, results_de, marker=:o)
plot!(p, sol.t, abs.(results_de .- exact_Nt.(sol.t)), label="error")